In [11]:
from jobspy import scrape_jobs
import pandas as pd
from datetime import date
import csv

# Your configs here

In [ ]:
years_old = 1000
result_wanted =20
job_title = '.net'
Locations = {
    "Netherlands": result_wanted,
    "Germany": result_wanted,
    "Belgium": result_wanted,
    "Austria": result_wanted,
    "Australia": result_wanted,
    "Canada": result_wanted,
    "France": result_wanted,
    "Ireland": result_wanted,
    "Italy": result_wanted,
    "Spain": result_wanted,
    "UK": result_wanted
}
job_sefixes = ["relocation", "relocate", "visa sponsorship"]


In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [106]:

# BEGIN: Combine all results into a single DataFrame
all_results = pd.DataFrame()

for location, results in Locations.items():
        for suffix in job_sefixes:
            try:
                temp_df = scrape_jobs(
                site_name=[
                        "Glassdoor",
                        "indeed",
                        "LinkedIn",
                        # "zip_recruiter"
                    ],
                    search_term=f"{job_title} {suffix}",
                    location=location,
                    results_wanted=results,
                    country_indeed=location,
                    hours_old=years_old,
                )
            except Exception as e:
                 continue
            all_results = pd.concat([all_results, temp_df], ignore_index=True)

print(f"Found {len(all_results)} jobs")
# print(all_results.head())


2024-09-18 19:09:28,837 - JobSpy - INFO - Indeed search page: 1
2024-09-18 19:09:28,839 - JobSpy - INFO - LinkedIn search page: 1
2024-09-18 19:09:30,147 - JobSpy - INFO - Indeed finished scraping
2024-09-18 19:09:30,726 - JobSpy - INFO - Glassdoor search page: 1
2024-09-18 19:09:34,335 - JobSpy - INFO - LinkedIn search page: 2
2024-09-18 19:09:34,990 - JobSpy - INFO - Linkedin finished scraping
2024-09-18 19:09:41,713 - JobSpy - INFO - Glassdoor finished scraping
2024-09-18 19:09:41,853 - JobSpy - INFO - Indeed search page: 1
2024-09-18 19:09:41,853 - JobSpy - INFO - LinkedIn search page: 1
2024-09-18 19:09:42,449 - JobSpy - INFO - Indeed finished scraping
2024-09-18 19:09:47,648 - JobSpy - INFO - Glassdoor search page: 1
2024-09-18 19:09:48,213 - JobSpy - INFO - LinkedIn search page: 2
2024-09-18 19:09:48,840 - JobSpy - INFO - Linkedin finished scraping
2024-09-18 19:09:49,443 - JobSpy - INFO - Glassdoor finished scraping
2024-09-18 19:09:49,580 - JobSpy - INFO - Indeed search page: 

Found 899 jobs


In [109]:
distinct_jobs = all_results.drop_duplicates(subset=['title', 'company'])

first_dict_words = ["c#", " .net ", "asp.net", "dotnet", "dot net", "csharp", "c sharp", "c-sharp",]

second_dict_words = ["reloc", "visa" , "sponsor", "relocate", "relocation"]

def contains_both_word_sets(description, first_words, second_words):
    if pd.isna(description):
        return False
    description_lower = description.lower()
    return any(word in description_lower for word in first_words) and any(word in description_lower for word in second_words)

distinct_jobs = distinct_jobs[distinct_jobs['description'].apply(lambda x: contains_both_word_sets(x, first_dict_words, second_dict_words))]

distinct_jobs = distinct_jobs.sort_values(by='date_posted', ascending=False)
print(f"distintcs are: {len(distinct_jobs)}")

distintcs are: 64


In [110]:
distinct_jobs.to_csv(f"jobs-{date.today()}.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_excel
# print(distinct_jobs)